In [1]:
import numpy as np
import math

# Invariant: S = [sij] is doubly stochastic (should this be checked in the function?) and 0 >= sij >= 1

S = np.array([[1/6, 2/3, 0, 1/6],
             [1/3, 1/6, 1/2, 0],
             [1/3, 1/6, 1/6, 1/3],
              [1/6, 0, 1/3, 1/2]])

Static = S

In [2]:
def more_0(S, S1, Shape):
    S_z = 0
    S1_z = 0
    
    for i in range(Shape[0]):
        for j in range(Shape[1]):
            if S[i][j] == 0: S_z += 1
            if S1[i][j] == 0: S1_z += 1
    
    return S1_z > S_z

In [3]:
def find_min_pos(S):
    int_max = 999999999999999999999999
    min_pos = int_max
    shape = S.shape
    for i in range(shape[0]):
        for j in range(shape[1]):
            c = S[i][j]
            if c < min_pos and c > 0:
                min_pos = c
    if min_pos is int_max: 
        raise Exception('No minimum positive coefficient in this matrix')
    else: 
        return min_pos

In [4]:
def Birkhoff(S):
    S01 = np.zeros(S.size)
    Shape = S.shape
    S01.shape = Shape

    coeffs = np.empty(0)
    perms = []
    while not np.array_equal(S, np.zeros(S.shape)):
        # Creating S01, S but all non-zero values are made into 1's
        for i in range(Shape[0]):
            for j in range(Shape[1]):
                S01[i][j] = math.ceil(S[i][j])

        # Permutation matrix can be <= S01, so I make it equal for simplicity
        perm_mat = np.array(S01)

        # Smallest coefficient in S is coefficient to permutation matrix in final step
        c = find_min_pos(S)

        # Final Step: S new = S old - c * Perm Mat
        Snew = S - c*perm_mat

        # Check to be sure that is a matrix having at least one more zero entry than S
        if not (more_0(S, Snew, Shape)):
            raise Exception('Less or equal number of zeros in Snew than in S')

        perms.append(perm_mat)
        coeffs = np.append(coeffs, c)
        S = Snew

    perms = np.array(perms)
    return perms, coeffs

In [5]:
def tester(S):
    perms, coeffs = Birkhoff(S)
    m = np.zeros(S.shape)
    for i in range(len(coeffs)):
        m = np.add(m, coeffs[i]*perms[i])
    if not np.allclose(m, S):
        print('Input: \n', S)
        print('Output: \n', m)
        print('Difference: \n', np.subtract(S, m))
        raise Exception('Arrays not Equal')

In [6]:
def DS_Generator(n):
    x = np.random.random((n,n))
    
    rsum = None
    csum = None

    while (np.any(rsum != 1)) | (np.any(csum != 1)):
        x /= x.sum(0)
        x = x / x.sum(1)[:, np.newaxis]
        rsum = x.sum(1)
        csum = x.sum(0)
        
    return x

In [10]:
for i in range(2, 20):
    print(i)
    m = DS_Generator(i)
    assert(np.allclose(m.sum(0), np.ones(i)))
    tester(DS_Generator(i))

2
3
4
5
6


KeyboardInterrupt: 